In [2]:
import sys
sys.path.append('../../')

# Optimization through Computational Evolution

So in the first tutorial we learned how to create didgeridoo geometries and how to analyze their sonic properties. In the second tutorial, we learned how to generate geometries from parametric shapes. In this tutorial, we will learn how to optimize the shapes through computational evolution.

In the first tutorial we defined a didgeridoo geometry and computed its sonic properties. In this tutorial, we want to reverse this: We define sonic properties and then come up with a didgeridoo geometry that has these properties. Because this is 1000x times more interesting, right?
ted
First I need to explain computational evolution and I try to make this as non-mathematical as possible.

Lets say we want to compute a Didgeridoo with a drone frequency of 73.4 Hz (which is D1). So how would you do this using the tools from tutorial 1? You would start with some kind of geometry. Maybe you already have an idea about didgeridoo acoustics, so you add a bell or make it cylindrical or you start with any other basic shape. Then you run the acoustical simulation and find out that that your didgeridoo has drone frequency for example 80 Hz. Then you change the shape. Whenever you get closer to your target (lets say you get a geometry with drone frequency 75 Hz), then you use this geometry and modify it further, until you are closer to the target.

### Loss Function

Computational evolution is very similar, except that is is automatic. It is a mathematical optimization algorithm. Now we need to get a bit mathematical, excuse me :) So it is an optimization algorithm, but what exactly do we want to optimize? We want to optimize the difference between our target properties and the properties our best geometry has. In DidgeLab, we formulate this differene as the loss function. The loss function defines how far we are away from our target. So, in above example, we could define the loss function as `loss(geo) = | drone_frequency(geo) - 73.4 Hz |`. So the loss function is the difference between the frequency of our didgeridoo and the target frequency. The `|` means that we use the absolute value: So in case our didgeridoo has drone frequency 50 Hz, then `drone_frequency(geo) - 73.4 Hz` would be negative. The absolute function turns all negative values positive. Using a loss function we can compare two didgeridoo shapes with each other and determine which one is closer to the target. The geometry that produces a lower loss is closer to the target.

Loss functions can be much more complicated than this. We can say that we want a bell size between 65 and 75mm, a certain drone frequency, certain overblow frequencies, we can add loss if the didgeridoo gets narrower instead of wider along the bore, and much more. The loss function is super important for the artificial evolution.

Computational evolution usually uses a fitness function, that determines which of two individuals (which of two didgeridoo shapes) is fitter or better. For reasons I dont remember anymore we use a loss function, so we determine which of two shapes is worse. In the end it really does not matter if you use fitness or loss, so for DidgeLab we go with loss.

## Computational evolution

So our goal is still the optimization of a didgeridoo geometries. Now we can compare two geometries which each other and find out which of the two is closer to the goal. So how exactly do we utilize that to come up with a shape that is close to the goal? First I explain the basic concept and than I explain the actual implementation, which is a bit more complex than the basic concept.

So for the basic concept, we start with a parametric shape (you remember these from tutorial 2?). This parametric shape already has some of our expectations embedded. E.g., we know that our didgeridoo should have a certain mouth piece size, a bell, and more. Also it has a set of parameters (e.g. bell size, length and others). The parameter set determines the geometry which determines the sonic properties. In computational evolution we have the notion of the "fittest" individual or the fittest parameter set, which is the individual with the smallest value of the loss function. When we begin, we have only a single individual so this is the fittest individual.

Now we randomly mutate this individual. For each mutant we compute the loss. If the loss is smaller than the loss of the fittest individual, than this mutant becomes the next fittest individual. Lets say we run this process over 1000 mutations. Each of these mutations is called a generation. At each of these 1000 generations, we mutate the fittest individual. Over the course of 1000 generations, the loss will never become bigger. The loss should always become smaller. And hopefully at the end of 1000 mutations, our geometry should have the properties which we defined over the loss.

So here you should take a deep breath and see if you understood this basic concept. Can you understand why this algorithm leads to geometries that have produce a low loss and can you understand how the loss function relates to the didgeridoo properties that we want to generate?

The actual implementation of computational evolution is a bit more complex. First of all, we do not do this one time, but several, usually 10, times. So in the end of the process we have 10 fittest individuals to choose from. We call the group of 10 mutants, that we mutate in parallel, the mutant pool.

The evolution has two phases, the exploration phase and the fine tuning phase. In the exporation phase, the mutation changes all parameters in a drastical way. In this way we try to come up with shapes that are close to what we want. In the fine tuning phase, the mutation changes only single parameters and it does not change them so drastically. In the fine tuning phase, we want to push the shapes closer to their target.

To better utilize parallelization and multiple CPU cores, each generation has a generation size, of e.g. 100. So each fittest individual in the mutant pool is mutated 100 times. Then these 100 mutations are compared to each other and the next fittest individual is selected. This determines the father shape for the next generation.

## Practical example

Enough theory. Lets define a loss function and play around with it. First we start with one of the shapes. We print the parameters of MbeyaShape.

In [3]:
from cad.calc.parameters import MbeyaShape
from cad.cadsd.cadsd import CADSD

parameters = MbeyaShape()
parameters

                  name    value     min      max  mutable
0             l_gerade  1000.00  500.00  1500.00    False
1             d_gerade     1.05    0.90     1.20    False
2   n_opening_segments     4.00    0.00     8.00    False
3     opening_factor_x     0.00   -2.00     2.00    False
4     opening_factor_y     0.00   -2.00     2.00    False
5       opening_length   850.00  700.00  1000.00    False
6           d_pre_bell    45.00   40.00    50.00    False
7               l_bell    35.00   20.00    50.00    False
8             bellsize    17.50    5.00    30.00    False
9         add_bubble_0     0.50    0.00     1.00    False
10     bubble_height_0     0.50    0.00     1.00    False
11        bubble_pos_0     0.50    0.00     1.00    False
12      bubble_width_0   150.00    0.00   300.00    False

And here are the initial resonant frequencies of the shape. Remember, the first resonant frequency is the drone note, the 2nd is the first toot.

In [4]:
geo = parameters.make_geo()
cadsd = CADSD(geo)
cadsd.get_notes()

,freq,impedance,rel_imp,note-number,cent-diff,note-name
1607,61.9,2.875512e+07,1.000000,-34,-4.609349,B1
146,147.0,5.573990e+06,0.193843,-19,-1.975158,D2
229,230.0,5.641366e+06,0.196187,-11,23.043595,A#3
323,324.0,2.158769e+06,0.075074,-5,29.811653,E3
409,410.0,1.914547e+06,0.066581,-1,22.255537,G#3
504,505.0,1.116385e+06,0.038824,2,-38.535837,B4
591,592.0,9.217152e+05,0.032054,5,-13.712383,D4
694,695.0,7.223093e+05,0.025119,8,8.588655,F4
801,802.0,6.221572e+05,0.021636,10,-39.318456,G4
900,901.0,8.624649e+05,0.029993,12,-40.828299,A5


Next we define a loss function. Lets say we want to make a Didgeridoo form that has a drone note of D and an octave toot. Also we want to ensure that the first three peaks have an impedance of more than 4e+06 because a peak does not need tuning alone, it also needs a certain impedance to be playable.

So here is the source code:

In [5]:
from cad.calc.loss import LossFunction, TootTuningHelper, diameter_loss, single_note_loss
from cad.calc.conv import note_to_freq
import tqdm as tqdm
from cad.calc.parameters import FinetuningParameters

# first we define the loss function
class OptimizeOpenDidgeridooLoss(LossFunction):

    def __init__(self):
        
        self.target_notes = [-31, -19]
        self.target_freqs = [note_to_freq(note) for note in self.target_notes]
        self.min_impedance = 4e+6
    
    # Every loss function implements the get_loss method that computes the loss 
    # for a geometry. Do not mind the context here, we do not need it.
    def get_loss(self, geo, context=None):
        
        # we do compute the cadsd directly, but we get it from the geometry.
        # in case we need it again, we do not need to compute it a 2nd time
        # but can use it from the cache in the geo object
        cadsd = geo.get_cadsd()
        notes = cadsd.get_notes()

        # if we have less than 3 resonant peaks then return a super high loss
        if len(notes) < 3:
            return 1000
        
        tuning_loss = 0
        for i in range(len(self.target_freqs)):
            # we compute the squared error for deviations from the target frequency
            diff = notes.freq.iloc[i] - self.target_freqs[i]
            tuning_loss += diff * diff
            
        volume_loss = 0
        for i in range(len(self.target_freqs)):
            imp = notes.impedance.iloc[i]
            if imp < self.min_impedance:
                diff = 1+(self.min_impedance-imp)/self.min_impedance   # this is a number between 1 and 2, with 1 meaning no deviation and 2 meaning super high deviation from our goal
                diff *= 20    # we want to put a high penalty to this
                volume_loss += diff

        # we return multiple loss values as a dictionary. the evolution considers only the "loss" key. the other keys can
        # give an insight in how the different parts of the loss influence the evolution.
        loss={
            "loss": tuning_loss + volume_loss,
            "tuning_loss": tuning_loss,
            "volume_loss": volume_loss,
        }
        return loss

loss = OptimizeOpenDidgeridooLoss()

First we defined the loss function. Then we created the parameters for the computational evolution. We can see that the FinetuningParameters define 18 parameters, 2 for each segment of the open didgeridoo geometry. The first two parameters are immutable, because we do not want the mouthpiece position to be anywhere alse than 0 and the diameter of the mouthpiece should stay at 32mm. All the other parameters can be changed within a certain region.

Next we perform an evolution. Therefore we define a processing pipeline. This pipeline consists of a single step only, the finetuning step. In this example we do not perform the exploration phase, because our shape is already almost where it should be.

In [6]:
from cad.calc.pipeline import Pipeline, FinetuningPipelineStep, ExplorePipelineStep
from cad.calc.mutation import MutantPool, FinetuningMutator, ExploringMutator
from cad.ui.evolution_ui import EvolutionUI
from cad.ui.evolution_progress_bar import EvolutionProgressBar

# our mutant pool contains only a single mutant. we do not want to mutate several mutants
mutant_pool=MutantPool.create_from_father(parameters, 1, loss)

# here we define a processing pipeline that consists of fine tuning only
pipeline=Pipeline()

n_generations=50
n_generation_size=10

pipeline.add_step(ExplorePipelineStep(ExploringMutator(), loss, mutant_pool, n_generations=n_generations, generation_size=n_generation_size))
pipeline.add_step(FinetuningPipelineStep(FinetuningMutator(), loss, n_generations=n_generations, generation_size=n_generation_size))

# this is a simple progress bar that shows the number of generations that has passed
# we pass the number of generations as a parameter. There are two pipeline steps, each has n_generations, 
# so in total there are n_generations*2 pipeline steps.
EvolutionProgressBar(n_generations*2)

# start the pipeline
final_mutant_pool=pipeline.run()

100%|██████████| 100/100 [01:05<00:00,  1.52it/s]


Lets print the notes and the didgeridoo shape. We are not super close to the target frequencies. But `n_generations` and `n_generation_size` are small. The higher these numbers the closer we will get to the target and the longer the skript runs.

In [7]:
from cad.ui.visualization import DidgeVisualizer

final_geo=final_mutant_pool.get(0).geo
final_geo.get_cadsd().get_notes()

DidgeVisualizer.vis_didge(final_geo)

,freq,impedance,rel_imp,note-number,cent-diff,note-name
1731,74.3,3.187289e+07,1.000000,-31,-20.716711,D1
146,147.0,4.491754e+06,0.140927,-19,-1.975158,D2
266,267.0,9.901732e+06,0.310663,-9,-35.203462,C3
366,367.0,1.164238e+06,0.036528,-3,14.068153,F#3
463,464.0,3.681287e+06,0.115499,1,8.054462,A#4
578,579.0,7.903697e+05,0.024798,5,24.728211,D4
664,665.0,1.420837e+06,0.044578,7,-15.020980,E4
797,798.0,7.189079e+05,0.022555,10,-30.662267,G4
853,854.0,7.216138e+05,0.022640,11,-48.079055,G#4
998,999.0,6.313005e+05,0.019807,14,-19.577385,B5
